<a href="https://colab.research.google.com/github/Farhanmohamedkn/Reinforcement_Learning/blob/master/Deep_Reinforcement_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building  Deep Reinforcement Learning model with OpenAI Gym and Python

In [1]:
import gym
import random

# 1. Test Random Environment with OpenAI Gym

In [2]:
env = gym.make('CartPole-v1')
states = env.observation_space.shape[0]
actions = env.action_space.n

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


Allowed actions by the cart pole

In [3]:
actions


2

Available states

In [4]:
states

4

# Lets build the random Environment

In [5]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render() #render not working anymore in collab try in notebook
        action = random.choice([0,1]) # take random action going right or left
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

/usr/local/lib/python3.10/dist-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Episode:1 Score:12.0
Episode:2 Score:15.0
Episode:3 Score:9.0
Episode:4 Score:25.0
Episode:5 Score:18.0
Episode:6 Score:34.0
Episode:7 Score:22.0
Episode:8 Score:15.0
Episode:9 Score:20.0
Episode:10 Score:27.0


so the  results are not great by taking random actions
Lets introduce a deep learning model with keras that can select better actions for us sow we can get more reward and keep the cart pole straight

# 2. Create a Deep Learning Model with Keras

import numpy as np

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.optimizers import Adam

In [6]:
!pip install keras-rl2

import numpy as np

from tensorflow.keras import models

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.optimizers import Adam

In [7]:
def build_model(states, actions):
    model = Sequential() # we build an sequential model not convolutional
    model.add(Flatten(input_shape=(1,states))) #input layer 1 flat node and states
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear')) #output is some action left or right
    return model

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Lets use our sequential model

In [8]:
model = build_model(states, actions) #passed the actions and states

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 4)                 0         
                                                                 
 dense (Dense)               (None, 24)                120       
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770 (3.01 KB)
Trainable params: 770 (3.01 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 3. Build Agent with Keras-RL

Now we use the model and train it with Keras RL(normally we use data in other types of machine learning here we training the agent

from rl.agents import DQNAgent (we have different types of agent available in keras)

from rl.policy import BoltzmannQPolicy (in RL we have value based and ploicy based learning

from rl.memory import SequentialMemory (for DQN agent we need some sequential memory)




In [ ]:
!python --version

from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
# we building the agent

def build_agent(model, actions): #passing the model and actions to the agent
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, #our agent we passed the model and memory and policy to it
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

# Lets initiate the agent and compile it

In [ ]:
dqn = build_agent(model, actions) #our agent
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

# Great ...Lets print the reward and see how much our RL agent earned

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

# Lets test the agent to evaluate the performance

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

# 4. Reloading Agent from Memory

Save the model so we can use it later if we want

In [ ]:
# dqn.save_weights('dqn_weights.h5f', overwrite=True) # save it our Storage

# Lets delete our env model and eveything then rebuild it

In [ ]:
# del model
# del dqn
# del env

In [ ]:
# env = gym.make('CartPole-v0')
# actions = env.action_space.n
# states = env.observation_space.shape[0]
# model = build_model(states, actions)
# dqn = build_agent(model, actions)
# dqn.compile(Adam(lr=1e-3), metrics=['mae'])

# lets load our model again from the storage

In [ ]:
# dqn.load_weights('dqn_weights.h5f')

# Lets test the RL model again

In [ ]:
# _ = dqn.test(env, nb_episodes=5, visualize=False)